In [5]:
from collections.abc import MutableMapping
class map_base(MutableMapping):
    class _item:
        def __init__(self,k,v):
            self._key=k
            self._value=v
    def __eq__(self,other):
        return self._key==other._key
    def __ne__(self,other):
        return not(self==other)
    def __lt__(self,other):
        return self._key<other._key

In [41]:
class Unsorted_tablemap(map_base):
    def __init__(self):
        self._table=[]
    def __getitem__(self,k):
        for item in self._table:
            if(item._key==k):
                return item._value
        raise KeyError('KeyError: '+ k)
    def __setitem__(self,k,v):
        for item in self._table:
            if(k==item._key):
                item._value=v
                return
        self._table.append(self._item(k,v))
    def __delitem__(self,k):
        for j in range(len(self._table)):
            if(k==self._table[j]._key):
                self._table.pop(j)
                return
        raise KeyError('Key Error'+k)
    def __len__(self):
        return len(self._table)
    def __iter__(self):
        for item in self._table:
            yield item._key

In [7]:
M=Unsorted_tablemap()
M['l']=1
M[1]=3
M[100]='h'
M[(6,7)]='p'
L=[7,8]
M[L]=10
print(M[L])
L.append(7)
print(M[L])

'''No error in using list in the unsortedtablemap method'''
#d={} 
#d[[6,7,4]]=1 returns 'mutable' type error
print(M[100])
print(M[(6,7)])

10
10
h
p


In [42]:
from random import randrange
class HashMapBase(map_base):
    def __init__(self,cap=11,p=109345121):
        self._table=[None for i in range(cap)]
        self._n=0
        self._prime=p
        self._scale=1+randrange(p-1)
        self._shift=randrange(p)
    def _hash_function(self,k):
        return (hash(k)*self._scale+self._shift)%self._prime%len(self._table)#Linear Probing
    #for quadratic probing,
    '''while(True):
    i+=1
    g=(hash(k)+i^2)%len(self._table)
    if(self._is_available(g)):
    return g
    This can only be implemented in open addressing hashing(and not in chaining). 
    '''
    def __len__(self):
        return self._n
    def __getitem__(self,k):
        j=self._hash_function(k)
        return self._bucket_getitem(j,k)
    def __setitem__(self,k,v):
        j=self._hash_function(k)
        self._bucket_setitem(j,k,v)
        if(self._n>len(self._table)//2):
            self._resize(2*len(self._table))
        return
    def __delitem__(self,k):
        j=self._hash_function(k)
        self._bucket_delitem(j,k)
        self._n-=1
    def _resize(self,k):
        #print(k)
        old=list(self.items())
        self._table=k*[None]
        self._n=0
        for k,v in old:
            self[k]=v

In [44]:
class ChainHashMap(HashMapBase):
    def _bucket_getitem(self,j,k):
        bucket=self._table[j]
        if(bucket[k] is None):
            raise KeyError('KeyError'+str(k))
        return bucket[k]
    def _bucket_setitem(self,j,k,v):
        if self._table[j] is None:
            self._table[j]=Unsorted_tablemap()
        oldsize=len(self._table[j])
        self._table[j][k]=v
        if(len(self._table[j])>oldsize):
            self._n+=1
    def _bucket_delitem(self,j,k):
        bucket=self._table[j]
        if(bucket is None):
            raise TypeError('keyerror'+str(k))
        del bucket[k]
    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:
                for key in bucket:
                    yield key

In [49]:
F=ChainHashMap()
from time import time
from random import randint
keys=[]
st=time()
for i in range(10000000):
    keys=randint(1,100000)
    values = randint(1,100000)
    F[keys]=values
print(time()-st,'secs')

166.56705498695374 secs


In [29]:
class ProbeHashMap(HashMapBase):
    _AVAIL=object()
    def _is_available(self,j):
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL
    def _find_slot(self,j,k):
        first_avail=None
        while(True):
            if(self._is_available(j)):
                if(first_avail is None):
                    first_avail=j
                if(self._table[j] is None):
                    return False,first_avail
            elif(k==self._table[j]._key):
                return True,j
            j=(j+1)%(len(self._table))
        return
    def _bucket_getitem(self,j,k):
        found,s=self._find_slot(j,k)
        if(not found):
            raise KeyError('Key Error'+repr(k))
        return self._table[s]._value
    def _bucket_setitem(self,j,k,v):
        found,s=self._find_slot(j,k)
        if(not found):
            self._table[s]=self._item(k,v)
            self._n+=1
        else:
            self._table[s]._value=v
        return
    def _bucket_delitem(self,j,k):
        found,s=self._find_slot(j,k)
        if(not found):
            raise KeyError('Key Error'+repr(k))
        self._table[s]=ProbeHashMap._AVAIL
        return
    def __iter__(self):
        for j in range(len(self._table)):
            if(not self._is_available(j)):
                yield self._table[j]._key

In [48]:
F=ProbeHashMap()
from time import time
from random import randint
keys=[]
st=time()
for i in range(10000000):
    keys=randint(1,10000)
    values = randint(1,100)
    F[keys]=values
print(time()-st,'secs')

140.82583832740784 secs


In [63]:
#Object at 'x' refers to the position in memory and id(x) returns the position as well
#
t=object()
t=object()
print(t)
print(hash(t),id(t))

107622436684 1721958986944
